In [1]:
import numpy as np
import pandas as pd
# from calibration.Calibration import Calibration 


In [2]:
# load rankings
reco_matrix = np.load("model_reco/vaecf/reco_matrix_vaecf_ml100k_100.npy")
print(reco_matrix.shape)

(943, 100)


In [3]:
import pickle

with open("model_reco/vaecf/score_dicts_vaecf_ml100k.pkl", "rb") as file:
    scores = pickle.load(file)
scores.__len__()


943

In [4]:
movies = pd.read_csv(
    "./data/ml-100K/i_id_mapping_genre.csv",
    sep="\t",
    names=[
        "item_id",
        "Name",
        "genres",
        "itemID"
    ],
    header=0,
    # encoding="latin-1",
)
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0
    
for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1


movies

,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,Kolya (1996),Comedy,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,L.A. Confidential (1997),Thriller|Mystery|Film-Noir|Crime,1,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,Heavyweights (1994),Children's|Comedy,2,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Legends of the Fall (1994),Romance|Western|War|Drama,3,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,Jackie Brown (1997),Crime|Drama,4,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1343,American Buffalo (1996),Drama,1343,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1344,A Chef in Love (1996),Comedy,1344,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1345,Nénette et Boni (1996),Drama,1345,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1346,Hollow Reed (1996),Drama,1346,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
users = pd.read_csv("./data/ml-100k/u_id_mapping_demographic_.csv", sep="\t")
users = users.sort_values(by="userID")
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
user_features_numpy = users.to_numpy()
users


,user_id,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
195,196,49,0,writer,55105,0,20
185,186,39,1,executive,00000,1,6
21,22,25,0,writer,40206,2,20
243,244,28,0,technician,80525,3,19
165,166,47,0,educator,55113,4,3
...,...,...,...,...,...,...,...
938,939,26,1,student,33319,938,18
935,936,24,0,other,32789,939,13
929,930,28,1,scientist,07310,940,17
919,920,30,1,artist,90008,941,1


In [6]:
item_ids = movies["itemID"].to_numpy()
item_ids

array([   0,    1,    2, ..., 1345, 1346, 1347])

In [38]:
# tahsin kheya
# last modified 23/01/2025
import pandas as pd
import os
import numpy as np

# import torch
import time
from logging import getLogger
import random
import numpy as np

from scipy.stats import entropy
# from mpi4py import MPI

# comm = MPI.COMM_WORLD
rank = 0


class GreedyCalibration(object):
    def __init__(self, config, movies, top_k, unique_genres, users, sensitive_attr, beta):
        self.top_k = top_k
        self.users_df = users
        self.users_np = users.to_numpy()
        self.beta = beta
        self.actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
        self.actual_genre_dist = self.actual_genre_dist.sort_values(by="userID")
        self.unique_genres = unique_genres
        self.i =movies
        self.i[unique_genres] = self.i[self.unique_genres].div(
            self.i[self.unique_genres].sum(axis=1), axis=0
        )
        self.sensitive_attr = sensitive_attr
        self.sensitive_compare_dist = []
        self.actual_distribution_sensitive = []

    def get_recom_distribution(self, reco, uid, compare_dist, alpha):
        reco = np.array(reco)
        selected_items = self.i[self.i['itemID'].isin(reco)]
        selected_items = selected_items.set_index('itemID').loc[reco].reset_index()
        selected_items = selected_items[unique_genres].to_numpy()
        indices = np.arange(len(reco))
        weights = 1 / (indices + 1) ** 0.1
        updated_genre_proportions = (1 - alpha) * selected_items + alpha * compare_dist
        updated_genre_proportions = weights[:, None] * updated_genre_proportions
        updated_genre_proportions = updated_genre_proportions.sum(axis=0)
        return updated_genre_proportions

    def compute_diversity_score(self, reco_items, uid, scores, b):
        alpha = 0.01
        userid = self.users_df.columns.get_loc("userID")
        sens_atr = self.users_df.columns.get_loc(self.sensitive_attr)
        filtered_rows = self.users_np[self.users_np[:, userid] == uid][0]
        current_user_sensitive_attr= filtered_rows[sens_atr]
        compare_dist = self.sensitive_compare_dist[current_user_sensitive_attr]
        # print(f"user id {uid} with gender {current_user_sensitive_attr} compare_dist {compare_dist} ")
        reco_dist = self.get_recom_distribution(reco_items, uid, compare_dist, alpha)  # sum wr(i)q˜(д|i),
        reco_dist = np.log(reco_dist)  # log sum wr(i)q˜(д|i),
        faireness_term = np.sum(compare_dist * reco_dist)
        sum_score = sum(scores[item] for item in reco_items)
        return (sum_score, faireness_term)

    def get_improved_reco(self, top_items, items, scores):
        self.get_sensitive_genre_dist()
        return self.get_new_recommendations(
            reco=top_items, scores=scores, all_items=items
        )

    def get_sensitive_genre_dist(self):
        actual_dist_sensitive = pd.merge(
            self.actual_genre_dist, self.users_df, on="userID"
        )
        sensitive_genre_weights_a = actual_dist_sensitive.groupby(self.sensitive_attr)[
            self.unique_genres
        ].mean()
        self.actual_distribution_sensitive = sensitive_genre_weights_a.sort_index()
        sensitive_compare_dist = self.actual_distribution_sensitive.sum()
        self.sensitive_compare_dist = self.actual_distribution_sensitive.apply(
            lambda r: sensitive_compare_dist - r, axis=1
        )
        nonzeroarray = np.full(len(self.unique_genres), 0.001)
        tradeoff = 0.01
        self.sensitive_compare_dist = tradeoff * nonzeroarray + (1-tradeoff) * self.sensitive_compare_dist #just to ensure we never get log 0 error
        self.sensitive_compare_dist = (
            self.sensitive_compare_dist[unique_genres].sort_index().to_numpy()
        )
        

    def normalize_scores(self, diversity_scores, b):
        div_scores = np.array(diversity_scores)
        scores = div_scores[:, 0]
        fairness = div_scores[:, 1]
        min_score,min_fairness = np.min(scores), np.min(fairness)
        max_score,max_fairness = np.max(scores), np.max(fairness)
        score_norm = (scores- min_score) /(max_score-min_score)
        fairness_norm = (fairness-min_fairness)/(max_fairness-min_fairness)
        fair_score_norm = (1 - b) * score_norm + b * fairness_norm
        fair_score_norm[div_scores[:, 0] == 0] = -9999
        return fair_score_norm

    def get_new_recommendations(self, reco, scores, all_items):
        """reco is 6040x50 and scores is 6040x3416"""
        b = self.beta  # beta for the fairness term
        # print(f"beta {b}")
        all_users = []
        top_k = self.top_k
        num_users = len(scores)
        # for u in range(rank*2,rank*2+2):
        upper_bound = min(num_users, rank * 25 + 25)
        
        for u in range(3, num_users):
            start_time = time.perf_counter()
            # remaining_items = list(range(10))
            remaining_items = reco[u]
            # print(remaining_items)
            u_calibrated = []
            for k in range(top_k):
                diversity_scores = [
                    self.compute_diversity_score(u_calibrated + [i], u, scores[u], b)
                    for i in remaining_items
                ]
                norm_diversity_scores = self.normalize_scores(diversity_scores, b)
                max_index = np.argmax(norm_diversity_scores)
                best_item = remaining_items[max_index]
                u_calibrated.append(best_item)
                remaining_items=np.delete(remaining_items, max_index)
            end_time = time.perf_counter()
            print(f"time taken {end_time-start_time}")
            print(f"user {u} u_calibrate {u_calibrated}")
            all_users.append(u_calibrated)

        return np.array(all_users)


In [39]:
config = {"user_genre_dist_file": "./data/ml-100k/pgui.csv"}
sensitive_attr = "Occupation_Code"
# movies = movie df
calibration = GreedyCalibration(config, movies,10, unique_genres, users, sensitive_attr, 0.9)
retVal = calibration.get_improved_reco(reco_matrix, list(item_ids), scores)


time taken 0.4611720829998376
user 3 u_calibrate [161, 47, 49, 389, 361, 118, 307, 430, 77, 407]
time taken 0.45772262499667704
user 4 u_calibrate [117, 673, 357, 184, 795, 252, 24, 325, 565, 742]
time taken 0.4591588749899529
user 5 u_calibrate [161, 78, 189, 254, 389, 361, 93, 357, 31, 321]
time taken 0.45348512500640936
user 6 u_calibrate [161, 49, 389, 321, 118, 307, 361, 500, 95, 407]
time taken 0.4602261670079315
user 7 u_calibrate [161, 49, 389, 321, 118, 93, 256, 77, 367, 407]
time taken 0.4556408329954138
user 8 u_calibrate [161, 78, 189, 273, 118, 389, 93, 256, 289, 430]


KeyboardInterrupt: 

In [ ]:
[[0.08129701 0.09984646 0.10186467 0.00437437 0.02813458 0.02732321
  0.003018   0.00602636 0.00491627 0.17567712 0.03694285 0.03532941
  0.02621846 0.03241395 0.02030809 0.03561443 0.01182731 0.26886745]
 [0.09949207 0.10652983 0.07843789 0.00750481 0.0212343  0.02824415
  0.00267132 0.00756791 0.00676386 0.16412515 0.04600522 0.05045142
  0.02637584 0.0358961  0.01568117 0.0390856  0.01119075 0.25274259]]

In [20]:
users[users["userID"]==25]

,user_id,Age,Gender,Occupation,ZipCode,userID,Occupation_Code
37,38,28,1,other,54467,25,13


In [ ]:
time taken 3.5609370830934495
user 3 u_calibrate [161, 47, 49, 389, 361, 118, 307, 430, 77, 407]
time taken 3.3222476250957698
user 4 u_calibrate [117, 673, 357, 184, 795, 252, 24, 325, 565, 742]
time taken 3.5916312499903142
user 5 u_calibrate [161, 78, 189, 254, 389, 361, 93, 357, 31, 321]


In [47]:
(0.45515195799816865-3.5609370830934495)/3.5609370830934495

-0.8721819713807551

In [17]:
a = np.array([[1,2,3,4],[5,6,7,8],[1,2,2,3]])
b=np.array([5,6,7,8])
c=np.array([1,2,2,3])
x = a.sum(axis=1)
x

array([10, 26,  8])

In [ ]:
m = movies
m[unique_genres] = movies[unique_genres].div(
        movies[unique_genres].sum(axis=1), axis=0
        )
m

In [ ]:
st = time.perf_counter()
u_calibrated = np.empty(1000)
current_index = 0
for i in range(1000):
    u_calibrated[current_index] = i
    current_index += 1
# u_calibrated = u_calibrated[:current_index]  # Trim if necessary
et = time.perf_counter()
print(et-st)


In [ ]:
np.save("occupation_9", retVal)

In [ ]:
users[3:10]

In [ ]:
s= time.time()
current_user_sensitive_attr = users.loc[
            users["userID"] == 4, "Gender"
        ].item()
e= time.time()
print(e-s)
current_user_sensitive_attr


In [ ]:
usersnp=users.to_numpy()

s= time.time()
userid = users.columns.get_loc("userID")
sens_atr = users.columns.get_loc("Gender")
filtered_rows = usersnp[usersnp[:, userid] == 4][0]
sens_atr= filtered_rows[sens_atr]
e= time.time()
print(e-s)

print("Original array:")
print(sens_atr)
print("\nFiltered rows where the 5th column is 4:")
print(filtered_rows)


In [ ]:
users

In [ ]:
p = np.array([0.1, 0.3, 0.6])
q = np.array([0.3, 0.4, 0.3])
# q˜(д|u) = (1 − α) · q(д|u) + α · p(д|u)
q_t = (1 - 0.01) * q + 0.01 * p
print(q_t)

In [ ]:
movies[793:794]

In [ ]:
movies[movies["genres"]==""]

In [ ]:
scores[0][1]

In [ ]:
sum(
    [
        0.0,
        0.01923077,
        0.0,
        0.11965812,
        0.0,
        0.02136753,
        0.0,
        0.00641026,
        0.0,
        0.02564101,
        0.5769231,
        0.02777778,
        0.01282051,
        0.0,
        0.0,
        0.0,
        0.01495726,
        0.0,
        0.17521365,
    ]
)

In [ ]:
item_index = movies.index.get_loc(0)
item_index

In [ ]:
reco = [1, 23, 4]
reco_2 = np.array(reco)
reco_2.flatten()

In [ ]:
nonzero_indices = np.where(p_dist != 0)

In [ ]:
nonzero_indices

In [ ]:
movies

In [ ]:
actual_genre_dist = pd.read_csv(
            os.path.join(config["user_genre_dist_file"]),
            sep="\t",
        )
actual_genre_dist = actual_genre_dist.sort_values(by="userID")
x = pd.merge(
            actual_genre_dist, users, on="userID"
        )
x = x.groupby(sensitive_attr)[
            unique_genres
        ].mean()
x

In [ ]:
total_sum = x.sum()
total_sum

In [ ]:

sum_except_self = x.apply(lambda row: total_sum - row, axis=1)
sum_except_self

In [ ]:

sum_except_self.sort_index(ascending=True).to_numpy()[0]

In [ ]:
print(int(users[users["userID"]==1]["Occupation_Code"]))

In [ ]:
occupation_code = users.loc[users["userID"] == 1, "Occupation_Code"].item()
occupation_code


In [ ]:
users[users["Occupation_Code"]==4]

In [ ]:
users

In [ ]:
0.5*0.99+0.01*0.26886745

In [ ]:
for i, genre in enumerate(unique_genres):
    print(i)
    print(genre)

In [ ]:
2*(0.2*0.01+0.99*0.02)

In [ ]:
(0.2*0.01+0.99*0.04)

In [ ]:
0.895958*0.000353